In [66]:
import numpy
import keras
import os
import tensorflow as tf
import pandas as pd
import cv2

from keras.preprocessing import image
# use mobile net - size is 15MB which is very low in comparison to VGG16(>500MB), InceptionV3(>500MB)
from keras.applications.mobilenet import MobileNet

from PIL import Image

In [33]:
def preprocess(img):
    
    width, height = img.shape[0], img.shape[1]
    
    img = image.array_to_img(img, scale=False)
    
    desired_width, desired_height = 224, 224
    
    if width < desired_width:
        desired_width = width
    if height < desired_height:
        desired_height = height
    
    start_x = np.maximum(0, int((width-desired_width)//2))
    
    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x + desired_width, height))
    
    img = img.resize((299, 299))
    
    img = image.img_to_array(img)
    
    return img / 255    
    

In [5]:
train_folder = "data/news_imgs/train"

In [6]:
files = os.listdir(train_folder)

In [44]:
files = list(map(lambda x: os.path.join(train_folder, x), files)) # load all the training images

In [8]:
# get started with just 20 images for training
train_images = files[0:20]

# preprocess images and load them
images = []
for item in train_images:
    img = image.load_img(item)
    img = image.img_to_array(img)
    preprocess(img)
    images.append(img)

In [67]:
# intialize the model
model = MobileNet(input_shape=(224, 224, 3), alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=False, weights='imagenet', input_tensor=None, pooling=None, classes=1000)

In [68]:
# compile the model with adam optimizer
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])

In [11]:
labels = pd.read_csv("data/news_imgs/training.csv") # load csv of text data provided

In [69]:
def convert_labels_to_numpy_array(items): 
    values = []
    for index, row in items.iterrows():

        cats = {'protest': True,
                'violence': True,
                'sign': True,
                'photo': True,
                'fire': True,
                'police': True,
                'children': True,
                'group_20': True,
                'group_100': True,
                'flag': True,
                'night': True,
                'shouting': True
               }


        array = []
        for cat in cats:
            try:
                f = float(row[cat])
                array.append(f)
            except ValueError:
                array.append(-1.0) # substitue for '-' values. Need to check with instructor how to handle them.

        narray = np.array(array)

        values.append(narray)

    return values

17227776/17225924 [==============================] - 2s 0us/step


             fname  protest violence sign photo fire police children group_20  \
0  train-00000.jpg        0        -    -     -    -      -        -        -   
1  train-00001.jpg        0        -    -     -    -      -        -        -   
2  train-00002.jpg        0        -    -     -    -      -        -        -   
3  train-00003.jpg        0        -    -     -    -      -        -        -   
4  train-00004.jpg        0        -    -     -    -      -        -        -   

  group_100 flag night shouting  
0         -    -     -        -  
1         -    -     -        -  
2         -    -     -        -  
3         -    -     -        -  
4         -    -     -        -  
